## **Data Exploration - Important Sanity Check Steps**

### 1. Initial Setup Checks
Before analyzing data, ensure that the dataset is properly loaded and structured.

- Check basic structure: Use `df.head()`, `df.tail()`, and `df.info()` to inspect the first and last few rows, data types, and overall shape.
- Check `df.sample()` to see a random sample.
- **Inspect Column Names (`.columns`)**
   - Check for **unwanted characters** such as `#`, `%`, `@`, etc.
   - Identify and **remove whitespaces** for easier column access.
   - Standardize **case formatting** (lowercase is generally easier to handle).
   - If necessary, rename columns systematically.
- Check index (`df.index`):
  - Ensure uniqueness (`df.index.is_unique`).
  - Identify index data type (string, numeric, timestamp) to check for consistency.
  - Reset the index if necessary (`df.reset_index(drop=True)`).
  - Set an appropriate column as an index if required (`df.set_index("column_name")`).

### 2. Data Quality Checks
Identify and fix common data issues to ensure accuracy.
- **Duplicates:** Count duplicate rows (`df.duplicated().sum()`) and remove if needed (`df.drop_duplicates()`).
- **Missing values:** Detect nulls (`df.isnull().sum()`) and assess impact.
- **Invalid values:** Look for impossible or inconsistent entries (e.g., negative prices, incorrect dates).
- **String formatting:** Standardize capitalization, trim whitespace.
- **Data types:** Ensure numbers are stored as numeric types (`df.dtypes`).

### 3. Data Distribution & Summary Statistics
Understand overall patterns and variations in the data.
- **Basic stats:** Use `df.describe()` for summary statistics like mean, median, min, and max. This is good to have some sense, we will see the pitfalls of relying on these later.
  
- **Histograms:** Visualize distributions (`df.hist()`). (Normal, Skewed?)

### 4. Outlier & Anomaly Detection
Find unusual values that could distort analysis:

- **Box plots:** Identify outliers visually (`sns.boxplot(x=df["column_name"])`).
- **IQR method:** Identify values outside `[Q1 - 1.5*IQR, Q3 + 1.5*IQR]` (`interquartile range(IQR) = Q3 - Q1`)
- **Domain checks:** Use subject knowledge to validate anomalies (e.g., a person’s height should not be 10 feet).

<!-- ### **5. Relationships & Correlations**
Examine connections between variables.
- **Correlation matrix:** Identify strong relationships (`df.corr()`).
- **Pairplots:** Visualize interactions (`sns.pairplot(df)`).
- **Group analysis:** Compare values across categories (`df.groupby("category")["column_name"].mean()`). -->




### Resources

Run the cell below to fetch the data in `data` dir

In [1]:
%%bash
set -euo pipefail

REPO_ZIP_URL="https://github.com/UCB-urban-data101/classActivity_data/archive/refs/heads/main.zip"
TMP_DIR="$(mktemp -d)"

# Prefer wget, otherwise curl
if command -v wget >/dev/null 2>&1; then
  wget -q -O "$TMP_DIR/repo.zip" "$REPO_ZIP_URL"
else
  curl -L -s -o "$TMP_DIR/repo.zip" "$REPO_ZIP_URL"
fi

unzip -oq "$TMP_DIR/repo.zip" -d "$TMP_DIR"
SRC_DIR="$TMP_DIR/classActivity_data-main"

rm -rf data
cp -R "$SRC_DIR/data" .

rm -rf "$TMP_DIR"
echo "Fetch complete."


Fetch complete.


**Important: Avoid changing your raw data directly. Create a copy and apply your sanity checks and cleaning on that.**

Read more: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

### Setting Up: Importing Libraries and Loading Data

We begin by importing the necessary Python packages. We use `pandas` for data handling, `numpy` for numerical operations. **We only need to import each package once, at the top of the file, and then we can use them across the code.**

Lets assume that we are starting from a new file here.

After import, we load our dataset from a CSV file, and create a `DataFrame` using pandas `.read_csv()` method

The data we use here is
[NYC apartment price](https://data.cityofnewyork.us/Housing-Development/apartment-cost-list/wnmh-ssrd/about_data)

In [2]:
import pandas as pd
import numpy as np


print('pandas version: ', pd.__version__)
print('-'*20)  # create a separator for better readability



# Load the dataset - create a variable called df that points to the DataFrame created by pd.read_csv()
apt_df = pd.read_csv("data/apartment_cost_list_20250217.csv")  # change to apt_df

# print the shape of your dataframe on top to have a referece point
print(f"Dataset loaded with {apt_df.shape[0]} rows and {apt_df.shape[1]} columns.")

pandas version:  2.2.3
--------------------
Dataset loaded with 19596 rows and 18 columns.



### Checking Index and Column Names and Cleaning Up
Before we start exploring, let's first inspect both axes (index and columns) to ensure consistency and avoid potential issues.

**A Note about Column Access**

   In Pandas, you can access a column as a Series using dot notation (`df.column_name`), but this only works if the column name:
   - Has **no whitespace**
   - **Does not conflict** with built-in attributes/methods (e.g., you cannot access a column called `"count"` with `df.count` since this will call the count  )

   **Tip:** It’s best to keep column names **lowercase** and replace spaces with underscores (`_`) for consistency and make sure you don't have Python or pandas reserved words as you column names.


In [3]:
apt_df.columns
# what are the immediate observations you have?

Index(['Job #', 'Borough', 'House #', 'Street Name', 'Block', 'Lot', 'Bin #',
       'Job Type', 'Community - Board', 'Curb Cut', 'Fully Permitted',
       'Initial Cost', 'Proposed Zoning Sqft', 'Horizontal Enlrgmt',
       'Vertical Enlrgmt', 'Enlargement SQ Footage', 'Zoning Dist1',
       'Job Description'],
      dtype='object')

In [4]:
apt_df.head()

,Job #,Borough,House #,Street Name,Block,Lot,Bin #,Job Type,Community - Board,Curb Cut,Fully Permitted,Initial Cost,Proposed Zoning Sqft,Horizontal Enlrgmt,Vertical Enlrgmt,Enlargement SQ Footage,Zoning Dist1,Job Description
0,200612337,BRONX,946,LEGGETT AVENUE,2685,50,2005109,A1,202,NaN,04/02/2001,$480000.00,0,NaN,NaN,0,C1-4,INTERIOR GUT REHABILITATION OF RESIDENTI AL AP...
1,200612337,BRONX,946,LEGGETT AVENUE,2685,50,2005109,A1,202,NaN,04/02/2001,$480000.00,0,NaN,NaN,0,C1-4,INTERIOR GUT REHABILITATION OF RESIDENTI AL AP...
2,200621336,BRONX,1216,BEACH AVENUE,3764,13,2024720,A1,209,NaN,NaN,$45000.00,0,Y,NaN,607,R5,APPLICATION FILED TO CONVERT APARTMENT T O A ...
3,200621336,BRONX,1216,BEACH AVENUE,3764,13,2024720,A1,209,NaN,NaN,$45000.00,0,Y,NaN,607,R5,APPLICATION FILED TO CONVERT APARTMENT T O A ...
4,200622424,BRONX,550,CAULDWELL AVENUE,2623,135,2091318,A2,201,NaN,06/02/2000,$75000.00,0,NaN,NaN,0,NaN,REPLACEMENT OF GAS PIPING FROM EXISTING METERS...


In [5]:
# if you want to see the positional index of the columns you can cast it as a Series
pd.Series(apt_df.columns)

0                      Job #
1                    Borough
2                    House #
3                Street Name
4                      Block
5                        Lot
6                      Bin #
7                   Job Type
8          Community - Board
9                   Curb Cut
10           Fully Permitted
11              Initial Cost
12      Proposed Zoning Sqft
13        Horizontal Enlrgmt
14          Vertical Enlrgmt
15    Enlargement SQ Footage
16              Zoning Dist1
17           Job Description
dtype: object

we can rename our columns using `rename()` method and passed it a dictionary of old and new names. This is handy if we want to apply multiple changes at the same time to a handful of column headers such as removing the `#` and updating the names. But what if we have many column names?

In [6]:
# previous method - using rename.
# df_renamed = df.rename(columns = {'Job #' : 'Job_num', 'House #': 'House_num', 'Bin #':'Bin_num'})

We can do the same thing, using replace() method for all the column names at once!
To do that, we update our `apt_df.columns`, by assigning it new values.

**Tip**: It's best **to avoid** renaming columns in place. Instead, create a new variable, inspect the results, and then assign it to your columns.


In [7]:
# here, we use multiple chained str.replace() - note that order matters. Try changing the order and see what changes

new_names = apt_df.columns.str.replace(' #', '_num').str.replace(' - ', '_') .str.replace(' ', '_').str.lower()

new_names

Index(['job_num', 'borough', 'house_num', 'street_name', 'block', 'lot',
       'bin_num', 'job_type', 'community_board', 'curb_cut', 'fully_permitted',
       'initial_cost', 'proposed_zoning_sqft', 'horizontal_enlrgmt',
       'vertical_enlrgmt', 'enlargement_sq_footage', 'zoning_dist1',
       'job_description'],
      dtype='object')

The code below is identical to above, just written vertically! (skip if you like)

I prefer writing vertically for the ease of read - it follows the one thought per line design pattern!
You can choose your style as you like

In [8]:

new_names = (
    apt_df.columns
    .str.replace(' #', '_num')
    .str.replace(' - ', '_') # this is for one problem making column `'community - board'`
    .str.replace(' ', '_')
    .str.lower()
)
new_names

Index(['job_num', 'borough', 'house_num', 'street_name', 'block', 'lot',
       'bin_num', 'job_type', 'community_board', 'curb_cut', 'fully_permitted',
       'initial_cost', 'proposed_zoning_sqft', 'horizontal_enlrgmt',
       'vertical_enlrgmt', 'enlargement_sq_footage', 'zoning_dist1',
       'job_description'],
      dtype='object')

 Note that here, using `regex` would have been an alternative choice (read: better choice!), but since they are more complex, we are not using them here

Before updating the columns of my DataFrame using the `new_names`, lets make sure that they are unique.
We can use `is_unique` attribute, that returns a Boolean.


In [9]:
new_names.is_unique

True

Everything looks good, we can now update our column names


In [10]:
apt_df.columns = new_names # updates my column names
apt_df.columns

Index(['job_num', 'borough', 'house_num', 'street_name', 'block', 'lot',
       'bin_num', 'job_type', 'community_board', 'curb_cut', 'fully_permitted',
       'initial_cost', 'proposed_zoning_sqft', 'horizontal_enlrgmt',
       'vertical_enlrgmt', 'enlargement_sq_footage', 'zoning_dist1',
       'job_description'],
      dtype='object')

###  Checking for Duplicates

Duplicate records can reduce the validity and reliability of data, introduce bias or inconsistency, waste storage space and computing resources, and complicate data processing and analysis steps. Here, we learn how to check if there are any duplicate rows and learn to remove them using pandas. We use the `duplicated()` method to identify duplicates which returns a Boolean Series and `drop_duplicates()` to remove them.

#### Determine duplicate rows in the dataset

`.duplicated(keep='first')`  by default, pandas keep the first instance of the duplicates and tags the rest with True

`.duplicated(keep='last')` keep the last one

`.duplicated(keep= False)` tags all duplicates - you may lose data using this.

**so `apt_df.duplicated()` is identical to `apt_df.duplicated(keep='first')`**

In [11]:
# create a new variable for your Boolean mask
duplicates = apt_df.duplicated()
duplicates

0        False
1         True
2        False
3         True
4        False
         ...  
19591    False
19592     True
19593    False
19594    False
19595     True
Length: 19596, dtype: bool

In [12]:
# when applying sum() to Booleans, each True counts as 1 and each False counts as 0
duplicates.sum()

np.int64(6086)

In [13]:
# we can use value_counts to get the counts of unique values.
apt_df.job_num.value_counts()

job_num
103696608    6
121715509    5
122731865    5
102354300    5
104144052    5
            ..
104658256    1
104657435    1
104657006    1
104656604    1
104656409    1
Name: count, Length: 13393, dtype: int64

In [14]:
# Investigate duplicate values by filtering for a specific job number
# This helps identify cases where 'job_num' appears more than once
apt_df[apt_df['job_num'] == 103696608]

,job_num,borough,house_num,street_name,block,lot,bin_num,job_type,community_board,curb_cut,fully_permitted,initial_cost,proposed_zoning_sqft,horizontal_enlrgmt,vertical_enlrgmt,enlargement_sq_footage,zoning_dist1,job_description
5688,103696608,MANHATTAN,1000,PARK AVENUE,1496,34,1046822,A2,108,NaN,05/14/2004,$60000.00,0,NaN,NaN,0,R8,Renovation of an existing apartment. General ...
5689,103696608,MANHATTAN,1000,PARK AVENUE,1496,34,1046822,A2,108,NaN,05/14/2004,$60000.00,0,NaN,NaN,0,R8,Renovation of an existing apartment. General ...
5690,103696608,MANHATTAN,1000,PARK AVENUE,1496,34,1046822,A2,108,NaN,NaN,$51000.00,0,NaN,NaN,0,NaN,Renovation of an existing apartment. HVAC and...
5691,103696608,MANHATTAN,1000,PARK AVENUE,1496,34,1046822,A2,108,NaN,NaN,$51000.00,0,NaN,NaN,0,NaN,Renovation of an existing apartment. HVAC and...
5692,103696608,MANHATTAN,1000,PARK AVENUE,1496,34,1046822,A2,108,NaN,NaN,$51000.00,0,NaN,NaN,0,NaN,Renovation of an existing apartment. HVAC and...
5693,103696608,MANHATTAN,1000,PARK AVENUE,1496,34,1046822,A2,108,NaN,NaN,$51000.00,0,NaN,NaN,0,NaN,Renovation of an existing apartment. HVAC and...


In [15]:
# notice after excluding duplicated rows the index labels are not sequential anymore
apt_df_nodup = apt_df.drop_duplicates()
apt_df_nodup.head()

,job_num,borough,house_num,street_name,block,lot,bin_num,job_type,community_board,curb_cut,fully_permitted,initial_cost,proposed_zoning_sqft,horizontal_enlrgmt,vertical_enlrgmt,enlargement_sq_footage,zoning_dist1,job_description
0,200612337,BRONX,946,LEGGETT AVENUE,2685,50,2005109,A1,202,NaN,04/02/2001,$480000.00,0,NaN,NaN,0,C1-4,INTERIOR GUT REHABILITATION OF RESIDENTI AL AP...
2,200621336,BRONX,1216,BEACH AVENUE,3764,13,2024720,A1,209,NaN,NaN,$45000.00,0,Y,NaN,607,R5,APPLICATION FILED TO CONVERT APARTMENT T O A ...
4,200622424,BRONX,550,CAULDWELL AVENUE,2623,135,2091318,A2,201,NaN,06/02/2000,$75000.00,0,NaN,NaN,0,NaN,REPLACEMENT OF GAS PIPING FROM EXISTING METERS...
6,200622433,BRONX,671,WESTCHESTER AVENUE,2628,1,2093934,A2,201,NaN,06/02/2000,$75000.00,0,NaN,NaN,0,NaN,REPLACEMENT OF GAS PIPING FROM EXISTING METERS...
8,200622442,BRONX,700,EAST 156 STREET,2628,1,2093932,A2,201,NaN,06/02/2000,$75000.00,0,NaN,NaN,0,NaN,REPLACEMENT OF GAS PIPING FROM EXISTING METERS...


In [16]:
# you can reset index to solve that.
# in this case we don't need to keep the old index labels so we can use drop=True.
# if drop=False, the old index will be added as a column to the dataframe

apt_df_nodup = apt_df_nodup.reset_index(drop=True)
apt_df_nodup.head()

,job_num,borough,house_num,street_name,block,lot,bin_num,job_type,community_board,curb_cut,fully_permitted,initial_cost,proposed_zoning_sqft,horizontal_enlrgmt,vertical_enlrgmt,enlargement_sq_footage,zoning_dist1,job_description
0,200612337,BRONX,946,LEGGETT AVENUE,2685,50,2005109,A1,202,NaN,04/02/2001,$480000.00,0,NaN,NaN,0,C1-4,INTERIOR GUT REHABILITATION OF RESIDENTI AL AP...
1,200621336,BRONX,1216,BEACH AVENUE,3764,13,2024720,A1,209,NaN,NaN,$45000.00,0,Y,NaN,607,R5,APPLICATION FILED TO CONVERT APARTMENT T O A ...
2,200622424,BRONX,550,CAULDWELL AVENUE,2623,135,2091318,A2,201,NaN,06/02/2000,$75000.00,0,NaN,NaN,0,NaN,REPLACEMENT OF GAS PIPING FROM EXISTING METERS...
3,200622433,BRONX,671,WESTCHESTER AVENUE,2628,1,2093934,A2,201,NaN,06/02/2000,$75000.00,0,NaN,NaN,0,NaN,REPLACEMENT OF GAS PIPING FROM EXISTING METERS...
4,200622442,BRONX,700,EAST 156 STREET,2628,1,2093932,A2,201,NaN,06/02/2000,$75000.00,0,NaN,NaN,0,NaN,REPLACEMENT OF GAS PIPING FROM EXISTING METERS...


In [17]:
#compare the shapes of apt_dfs after dropping duplicated rows
apt_df.shape, apt_df_nodup.shape

((19596, 18), (13510, 18))

### Counting Missing Values

Missing values can occur for various reasons (data entry errors, unrecorded information, etc.).

- **Count Missing Values:** Use `isnull()` and `sum()` to see how many missing values are in each column.


In [18]:
# Count missing values per column
missing_counts = apt_df_nodup.isnull().sum()
print("Missing values per column:")
display(missing_counts)

Missing values per column:


job_num                       0
borough                       0
house_num                     0
street_name                   0
block                         0
lot                           0
bin_num                       0
job_type                      0
community_board               0
curb_cut                  13501
fully_permitted            2735
initial_cost                  0
proposed_zoning_sqft          0
horizontal_enlrgmt        13404
vertical_enlrgmt          13336
enlargement_sq_footage        0
zoning_dist1               1462
job_description               0
dtype: int64

In [19]:
# dropna()?
# fillna()?


### Some Basic Filtering

#### Filter Only for Manhattan and Brooklyn

In [20]:
mnh_brk = apt_df_nodup['borough'].isin(['MANHATTAN', 'BROOKLYN'])
apt_mnh_brk = apt_df_nodup[mnh_brk]
apt_mnh_brk.shape

(12568, 18)

### Validating Variable Types and Value Ranges

Now we verify that each column has the correct data type and that the values are within expected ranges. This step helps ensure that data meant to be numeric isn’t stored as text, and that values like prices or percentages are realistic.


### Example:
- Check that a `price` column has no negative values.
- Ensure a `percentage` column has values only between 0 and 100.
- Check columns containing `date & time` data

In [21]:
# Display data types of all columns
print("Data types in the dataset:")
display(apt_df_nodup.dtypes)

Data types in the dataset:


job_num                    int64
borough                   object
house_num                 object
street_name               object
block                      int64
lot                        int64
bin_num                    int64
job_type                  object
community_board            int64
curb_cut                  object
fully_permitted           object
initial_cost              object
proposed_zoning_sqft       int64
horizontal_enlrgmt        object
vertical_enlrgmt          object
enlargement_sq_footage     int64
zoning_dist1              object
job_description           object
dtype: object

### Categorical Data:

Categorical data in pandas has their own type called `category`. Read more about pandas categorical data here: https://pandas.pydata.org/docs/user_guide/categorical.html

In [22]:
# Inspect a column that we expect to be categorical - How many categories do we have?
apt_df_nodup.job_type.value_counts(dropna=False)

job_type
A2    12531
A1      938
A3       41
Name: count, dtype: int64

In [23]:
apt_df_nodup.job_type.astype('category')

0        A1
1        A1
2        A2
3        A2
4        A2
         ..
13505    A1
13506    A1
13507    A1
13508    A2
13509    A2
Name: job_type, Length: 13510, dtype: category
Categories (3, object): ['A1', 'A2', 'A3']

#### Now Your Turn

In [24]:

# identify another column that need to be converted to category type - note that it is best to use this type if we dont have too many categories

apt_df_nodup.borough.astype('category')

0                BRONX
1                BRONX
2                BRONX
3                BRONX
4                BRONX
             ...      
13505    STATEN ISLAND
13506    STATEN ISLAND
13507    STATEN ISLAND
13508    STATEN ISLAND
13509    STATEN ISLAND
Name: borough, Length: 13510, dtype: category
Categories (5, object): ['BRONX', 'BROOKLYN', 'MANHATTAN', 'QUEENS', 'STATEN ISLAND']

### Date-Time Data

The `fully_permitted` column contains date information. We will convert it to the `datetime` data type using `.dt` and explore its functionalities, such as finding the minimum and maximum dates, extracting months and days, and more.

In [25]:
apt_df_nodup['fully_permitted'].head()

0    04/02/2001
1           NaN
2    06/02/2000
3    06/02/2000
4    06/02/2000
Name: fully_permitted, dtype: object

#### **Converting a Column to Datetime Format in Pandas**  

In [26]:
# we can create a new column to store a converted version of an existing column as a datetime data type:

apt_df_nodup['date_new'] = pd.to_datetime(apt_df_nodup['fully_permitted'], format = '%m/%d/%Y', errors='raise')
apt_df_nodup['date_new']

0       2001-04-02
1              NaT
2       2000-06-02
3       2000-06-02
4       2000-06-02
           ...    
13505          NaT
13506          NaT
13507          NaT
13508   2019-06-28
13509          NaT
Name: date_new, Length: 13510, dtype: datetime64[ns]

**What happened here?**

1. **`pd.to_datetime()`** → Converts a column to a proper date format.  
2. **`apt_df_nodup['fully_permitted']`** → The original column that contains date values as text.  
3. **`format='%m/%d/%Y'`** → Specifies the date format:
   - `%m` = Month (e.g., `01` for January)  
   - `%d` = Day (e.g., `15` for the 15th day)  
   - `%Y` = Year (e.g., `2024`)  
4. **`errors='raise'`** → If a date is invalid, it will **stop with an error** instead of converting it incorrectly.  


- Ensures the column is in **actual datetime format**, allowing sorting, filtering, and calculations.
- Enables **date-based operations** like finding the oldest and newest dates, extracting months, and calculating time differences.



#### Now, we can use all the `.dt` methods pandas provides

In [27]:
# Sort by date (latest to earliest)
apt_df_nodup = apt_df_nodup.sort_values(by='date_new', ascending=False)
apt_df_nodup.head()

,job_num,borough,house_num,street_name,block,lot,bin_num,job_type,community_board,curb_cut,fully_permitted,initial_cost,proposed_zoning_sqft,horizontal_enlrgmt,vertical_enlrgmt,enlargement_sq_footage,zoning_dist1,job_description,date_new
805,302221138,BROOKLYN,234,WASHINGTON AVENUE,1917,32,3055065,A1,302,NaN,04/16/2024,$75000.00,0,NaN,NaN,0,R6,Proposed to convert one class 'A' apartment an...,2024-04-16
1346,321084919,BROOKLYN,25,REMSEN STREET,247,15,3002026,A2,302,NaN,02/13/2024,$66000.00,0,NaN,NaN,0,R6,RENOVATION OF AN EXISTING RESIDENTIAL DUPLEX A...,2024-02-13
1695,321759744,BROOKLYN,1717,AVENUE N,6746,44,3181052,A2,314,NaN,12/04/2023,$72000.00,0,NaN,NaN,0,R7A,INTERIOR RENOVATION OF EXISITNG APARTMENT 1J A...,2023-12-04
2225,340842280,BROOKLYN,822,MARCY AVENUE,1818,46,3051409,A2,303,NaN,08/07/2023,$50000.00,0,NaN,NaN,0,R6B,INTERIOR RENOVATION OF EXISTING APARTMENT 1R O...,2023-08-07
11133,123417962,MANHATTAN,441,WEST 48TH STREET,1058,10,1026588,A2,104,NaN,07/18/2023,$70000.00,0,NaN,NaN,0,R8,FILING TO SUBMIT PLANS TO SHOW CORRECTIVE WORK...,2023-07-18


In [28]:
# Add a column for weekdays vs. weekends

apt_df_nodup['day_of_week'] = apt_df_nodup['date_new'].dt.day_name()
apt_df_nodup['is_weekend'] = apt_df_nodup['date_new'].dt.weekday >= 5  # True for Saturday (5) and Sunday (6)


In [29]:
apt_df_nodup[apt_df_nodup['is_weekend']] # Filtering for only weekend

,job_num,borough,house_num,street_name,block,lot,bin_num,job_type,community_board,curb_cut,...,initial_cost,proposed_zoning_sqft,horizontal_enlrgmt,vertical_enlrgmt,enlargement_sq_footage,zoning_dist1,job_description,date_new,day_of_week,is_weekend
12878,140982792,MANHATTAN,51,FIFTH AVENUE,569,5,1009275,A2,102,NaN,...,$72000.00,0,NaN,NaN,0,R10,RENOVATION OF APARTMENT #11E; DEMOLITION OF NO...,2021-01-02,Saturday,True
12862,140974408,MANHATTAN,219,EAST 7 STREET,390,42,1004722,A2,103,NaN,...,$50000.00,0,NaN,NaN,0,R8B,INTERIOR RENOVATION OF EXISTING RESIDENTIAL AP...,2021-01-02,Saturday,True
12892,140989152,MANHATTAN,217,EAST 82ND STREET,1528,9,1048646,A2,108,NaN,...,$50000.00,0,NaN,NaN,0,R8B,INTERIOR RENOVATION OF APARTMENT #3A ON 4TH FL...,2020-12-12,Saturday,True
12880,140983158,MANHATTAN,322,EAST 90 STREET,1552,38,1050128,A2,108,NaN,...,$80000.00,0,NaN,NaN,0,R8B,INTERIOR RENOVATION ON EXISITNG APARTMENTS 2A ...,2020-11-29,Sunday,True
12737,140926942,MANHATTAN,62,WEST 71ST STREET,1123,60,1028584,A2,107,NaN,...,$70000.00,0,NaN,NaN,0,R8B,CREATION OF DEDICATED ACCESSORY ROOMS AT CELLA...,2020-11-28,Saturday,True
12872,140978841,MANHATTAN,337,E 50TH STREET,1343,16,1039606,A2,106,NaN,...,$40000.00,0,NaN,NaN,0,R8B,INTERIOR RENOVATION OF APARTMENT 4E. MINOR LAY...,2020-11-28,Saturday,True
12879,140983130,MANHATTAN,322,EAST 90 STREET,1552,38,1050128,A2,108,NaN,...,$80000.00,0,NaN,NaN,0,R8B,INTERIOR RENOVATION ON EXISITNG APARTMENTS B &...,2020-11-28,Saturday,True
2188,340768716,BROOKLYN,389,8TH STREET,1000,64,3021991,A2,306,NaN,...,$65000.00,0,NaN,NaN,0,R6B,RENOVATE APARTMENT ON 3RD FLOOR IN AN EXISTING...,2020-10-17,Saturday,True
12754,140931918,MANHATTAN,150,EAST 93RD STREET,1521,51,1048364,A2,108,NaN,...,$80000.00,0,NaN,NaN,0,C1-8X,INTERIOR RENOVATION OF APARTMENT 10A AS SHOWN ...,2020-09-13,Sunday,True
12742,140928691,MANHATTAN,385,GRAND STREET,311,13,1077605,A2,103,NaN,...,$50000.00,0,NaN,NaN,0,R8,HEREWITH FILING TO RENOVATE EXISTING APARTMENT...,2020-09-12,Saturday,True


In [30]:
# Count Entries by Day of the Week
day_counts = apt_df_nodup['date_new'].dt.day_name().value_counts()
day_counts

date_new
Tuesday      2349
Wednesday    2314
Thursday     2253
Friday       2054
Monday       1770
Saturday       28
Sunday          7
Name: count, dtype: int64

In [31]:
# Find the earliest and latest dates

min_date = apt_df_nodup['date_new'].min()
max_date = apt_df_nodup['date_new'].max()
print(f"Min Date: {min_date.date()}\nMax Date: {max_date.date()}")

Min Date: 2000-01-28
Max Date: 2024-04-16


In [32]:
# Extract month to a separate column `.dt.month`

apt_df_nodup['perm_month'] = apt_df_nodup['date_new'].dt.month
apt_df_nodup['perm_month']

805       4.0
1346      2.0
1695     12.0
2225      8.0
11133     7.0
         ... 
13503     NaN
13505     NaN
13506     NaN
13507     NaN
13509     NaN
Name: perm_month, Length: 13510, dtype: float64

#### **Your Turn:** **Count Permits by Year**

Now, create a new column named **`year`** and extract the year from the `fully_permitted` date column.

Next, use `.value_counts()` on the `year` column to count the number of permits issued each year.


In [33]:
# Your code here

apt_df_nodup['year'] = apt_df_nodup['date_new'].dt.year

permits_per_year = apt_df_nodup['year'].value_counts()
display(permits_per_year, 10)

year
2019.0    640
2008.0    636
2017.0    631
2016.0    613
2007.0    610
2018.0    594
2005.0    588
2006.0    579
2010.0    547
2015.0    539
2004.0    518
2009.0    516
2011.0    516
2014.0    494
2003.0    493
2012.0    455
2013.0    423
2020.0    422
2002.0    405
2001.0    277
2000.0    153
2021.0    113
2022.0      8
2023.0      3
2024.0      2
Name: count, dtype: int64

10

### Columns that should be numerical

column `initial_cost` should have the numerical value. However, as we observed during data exploration, this column is not stored as a numeric type but as an `object`. A closer inspection reveals that it contains a mix of `strings` (numbers with `$`).
We can use the `.str.strip()` method to remove `$` and then convert it to float

In [34]:
#inspect how the results look

apt_df_nodup['initial_cost'].str.strip('$')

805      75000.00
1346     66000.00
1695     72000.00
2225     50000.00
11133    70000.00
           ...   
13503    50000.00
13505    70000.00
13506    80000.00
13507    80000.00
13509    50000.00
Name: initial_cost, Length: 13510, dtype: object

In [35]:
# we remove the dollar sign ($) from the initial_cost column and convert it to a numeric (float) type

apt_df_nodup['initial_cost'] = apt_df_nodup['initial_cost'].str.strip('$').astype(float)


1. **`.str.strip('$')`** → Removes the `$` symbol from each value.  
2. **`.astype(float)`** → Converts the cleaned values from strings to floating-point numbers.  

Now, `initial_cost` can be used for calculations like **sorting, averaging, and summing costs**.

#### **Now Your Turn:**  
#### Find the Most Expensive Construction Projects

In [36]:
# Your code here

apt_df_nodup.iloc[apt_df_nodup.initial_cost.sort_values(ascending=True).index]['initial_cost']

13123    56000.0
5702     44000.0
5441     80000.0
6664     45000.0
13189    83000.0
          ...   
3889     75000.0
2352     55000.0
271      80000.0
12772    60000.0
3801     40000.0
Name: initial_cost, Length: 13510, dtype: float64